### PART 03) 통계분석

## 1장. 가설 검정

### 1절. 상관분석

In [1]:
# 데이터 호출한 후 데이터프레임으로 변환
import pandas as pd
from sklearn.datasets import load_diabetes 

diabetes = load_diabetes()
data = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
data.info() # 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [2]:
# scipy.stats.pearsonr
from scipy.stats import pearsonr
pearsonr(x = data['age'], y = data['bmi'])

(0.18508466614655558, 9.076791865412659e-05)

In [3]:
# 단순한 상관계수의 산출은 데이터프레임객체.corr()로도 가능
data[['age', 'bmi']].corr()

,age,bmi
age,1.000000,0.185085
bmi,0.185085,1.000000


In [4]:
# scipy.stats.spearmanr
from scipy.stats import spearmanr
spearmanr(a = data['sex'], b = data['bmi'])

SpearmanrResult(correlation=0.09807947297621517, pvalue=0.03929011358104615)

In [5]:
# 단순한 상관계수의 산출은 데이터프레임객체.corr()로 가능
# corr(method = 'spearman')은 스피어만 상관계수를 산출함
data[['sex', 'bmi']].corr(method = 'spearman')

,sex,bmi
sex,1.000000,0.098079
bmi,0.098079,1.000000


---

### 2절. 정규성 검정

In [6]:
# 필요한 패키지 호출
import numpy as np

# 균일표본 생성
# (참고) np.random.random(size) : 균일 분포로 부터 size개의 len을 가지는 난수 생성
np.random.seed(2024) # 결과의 일관성을 위해 시드값 설정
x = np.random.random(10)

# 샤피로-윌크 검정 수행
from scipy.stats import shapiro
shapiro(x)

ShapiroResult(statistic=0.8969665765762329, pvalue=0.20285499095916748)

---

### 3절. 모평균과 모분산 검정

#### 1. 모평균 검정

#### 가. 단일표본 t-검정

In [11]:
# 필요한 패키지 호출
import numpy as np

# 몸무게 데이터 임의 생성
kg = np.array([75.5, 83.9, 75.7, 56.2, 73.4, 67.7, 79.0, 50.7, 58.4, 74.1, 65.1, 77.8, 48.1, 46.3])

# 표본 평균
np.mean(kg)

66.56428571428572

In [12]:
# 단일 표본 t-검정 실행
from scipy.stats import ttest_1samp
ttest_1samp(kg, 70)

Ttest_1sampResult(statistic=-1.0289933120202257, pvalue=0.3222484823978743)

#### 나. 대응표본 t-검정

In [4]:
# 남녀 몸무게 데이터를 임의로 생성
import pandas as pd
female = np.array([50.7, 58.4, 74.1, 65.1, 77.8, 48.1, 46.3])
male = np.array([75.5, 83.9, 75.7, 56.2, 73.4, 67.7, 79.0])

# 두 데이터의 차이의 평균
diff = female - male
np.mean(diff)

# 대응표본 t-검정 실행
from scipy.stats import ttest_rel
ttest_rel(female, male)

Ttest_relResult(statistic=-2.0784469330649724, pvalue=0.08291274205610197)

#### 다. 독립표본 t-검정

In [5]:
# 독립표본 t-검정 실행
from scipy.stats import ttest_ind
ttest_ind(female, male)

Ttest_indResult(statistic=-2.218664157777296, pvalue=0.04655012211056963)

#### 2. 모분산 검정

#### 가. 단일표본 검정

In [6]:
# 점수 데이터 임의 생성
import numpy as np
score = np.array([80.5, 60.2, 70, 87, 45, 91, 85])

# 검정통계량 = (n-1) ⁎ 표본분산 / 귀무가설에서 설정한 모분산
var0 = 1100 # 귀무가설에서 설정한 모분산

var = np.var(score, ddof = 1) # 표본 분산
print(var)

stat = (len(score)-1)*var/var0 # 검정 통계량
print(stat)

# 카이분포를 통해 직접 유의확률 계산
# 좌측 검정이므로 Pr[chisq2(자유도) < 검정통계량]으로 계산
from scipy.stats import chi2
chi2.cdf(stat, len(score)-1)

278.1033333333333
1.5169272727272727


0.041637780038918736

In [8]:
# (참고) 우측 검정일 경우, Pr[chisq2(자유도) > 검정통계량]으로 계산
1 - chi2.cdf(stat, len(score)-1)

0.9583622199610813

In [9]:
# (참고) 양측 검정일 경우, 2⁎좌측검정의 유의확률 또는 2⁎우측검정의 유의확률로 계산
# 여기서는 표본분산 < 모분산이므로 좌측검정의 유의확률에서 2배를 곱함
2 * chi2.cdf(stat, len(score)-1)

# 유의수준 5%에서 모분산이 1200보다 작다고 할 수는 있지만 통계적으로 다르다고 하기는 어렵다.

0.08327556007783747

#### 나. 분산비 검정

In [10]:
# 두 집단(a, b)에 대한 점수 데이터 임의로 생성
a = np.array([70, 80, 75, 65, 100, 98])
b = np.array([20, 100, 50, 94, 28, 80, 95, 30])

# 표본 분산 계산
var_a = np.var(a, ddof = 1)
var_b = np.var(b, ddof = 1)
print(var_a, var_b)

# 검정통계량 = 집단a의 분산/집단b의 분산(가설: a모분산 < b모분산)
stat = var_a/var_b
print(stat)

# 자유도 계산
df1, df2 = len(a)-1, len(b)-1
print(df1, df2)

# F분포를 통해 직접 유의확률 계산
# 좌측 검정이므로 Pr[F(자유도1,자유도2) < 검정통계량]으로 계산
# 좌측 검정인 이유? 가설: a모분산 < b모분산 <=> a모분산/b모분산 < 1)
from scipy.stats import f
pval = f.cdf(stat, df1, df2)
print(pval)

212.66666666666669 1138.4107142857142
0.18681014154026346
5 7
0.04153943037562959


In [11]:
# (참고) 우측 검정일 경우, Pr[F(자유도1,자유도2) > 검정통계량]으로 계산
1 - f.cdf(stat, df1, df2)

0.9584605696243704

In [12]:
# (참고) 양측 검정일 경우, 현재 검정통계량이 1보다 작으므로 아래와 같이 계산함
# Pr[F(자유도1,자유도2) < 검정통계량] + Pr[F(자유도2,자유도1) > 1/검정통계량]
f.cdf(stat, df1, df2) + (1 - f.cdf(1/stat, df2, df1))

# 유의수준 5%에서 a 집단의 모분산이 b 집단의 모분산보다 작다고 할 수 있지만 통계적으로 다르다고 하기는 어렵다.
# (참고) 검정통계량이 1보다 큰 경우 Pr[F(자유도1,자유도2) > 검정통계량] + Pr[F(자유도2,자유도1) <1/검정통계량]

0.08307886075125917

#### 다. Bartlett 검정

In [13]:
# 두 집단(a, b)에 대한 점수 데이터 임의로 생성
# 임의로 세 그룹의 점수 데이터를 생성
a = np.array([70, 80, 75, 65, 100, 98])
b = np.array([20, 100, 50, 94, 28, 80])
c = np.array([90, 97, 95, 94, 99, 100])

# Bartlett 검정 수행
from scipy.stats import bartlett
bartlett(a, b, c)

BartlettResult(statistic=15.6702722148674, pvalue=0.00039558846873743075)

#### 라. Levene 검정

In [15]:
# Levene 검정 수행
from scipy.stats import levene
levene(a, b, c)

LeveneResult(statistic=14.365736704446384, pvalue=0.00032713621045500125)

#### 3. 분산분석

#### 가. 일원배치 분산분석

In [16]:
# 데이터 호출한 후 데이터프레임으로 변환
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

df = pd.read_csv('data/예제/school_score.csv')
print(df)

one_way = ols('Final ~ C(School)', data = df).fit()

# anova_lm 함수를 사용해 분산분석
result = anova_lm(one_way)
print(result)

        id School Sex  Grade  Final
0        1      A   M      1   44.4
1        2      A   M      2   47.7
2        3      A   M      3   65.6
3        4      A   F      1   50.7
4        5      A   F      2   51.3
...    ...    ...  ..    ...    ...
2995  2996      C   M      2   84.1
2996  2997      C   M      3   81.6
2997  2998      C   F      1   83.2
2998  2999      C   F      2   92.8
2999  3000      C   F      3  100.0

[3000 rows x 5 columns]
               df         sum_sq        mean_sq            F  PR(>F)
C(School)     2.0  996939.237147  498469.618573  5722.221007     0.0
Residual   2997.0  261072.308290      87.111214          NaN     NaN


#### 나. 사후검정

In [17]:
# 필요 패키지 및 함수 호출
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# tunky의 다중비교 시행
tukey_result = pairwise_tukeyhsd(endog = df['Final'], groups = df['School'], alpha = 0.05)
print(tukey_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B  38.8464   -0.0 37.8677 39.8251   True
     A      C  38.4922   -0.0 37.5135 39.4709   True
     B      C  -0.3542 0.6728 -1.3329  0.6245  False
----------------------------------------------------


#### 다. 이원배치 분산분석

In [18]:
# two way ols 모형객체를 생성(교호작용 ◯ )
two_way = ols('Final ~ C(School) + C(Grade) + C(School):C(Grade)', data = df).fit()

# anova_lm 함수를 사용해 이원배치 분산분석
result = anova_lm(two_way)
print(result)

                        df         sum_sq        mean_sq            F  \
C(School)              2.0  996939.237147  498469.618573  5721.620556   
C(Grade)               2.0     324.472829     162.236415     1.862210   
C(School):C(Grade)     4.0     170.851257      42.712814     0.490274   
Residual            2991.0  260576.984204      87.120356          NaN   

                      PR(>F)  
C(School)           0.000000  
C(Grade)            0.155509  
C(School):C(Grade)  0.742912  
Residual                 NaN  


In [19]:
# two way ols 모형객체를 생성(교호작용×)
two_way = ols('Final ~ C(School) + C(Grade)', data = df).fit()

# anova_lm 함수를 사용해 이원배치 분산분석
result = anova_lm(two_way)
print(result)

               df         sum_sq        mean_sq            F    PR(>F)
C(School)     2.0  996939.237147  498469.618573  5725.518315  0.000000
C(Grade)      2.0     324.472829     162.236415     1.863479  0.155312
Residual   2995.0  260747.835461      87.061047          NaN       NaN
